In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import transforms
import os

from torchvision.models.segmentation.deeplabv3 import DeepLabHead, deeplabv3_resnet101
# from torchvision.datasets import Cityscapes
from torchvision.datasets import DatasetFolder

from dataloader import DataLoaderSegmentation


In [41]:
%load_ext autoreload
%autoreload 2

# the path should contain three folders -> leftlmg8bit and label_processed
path = '/Users/rajkrishnanv/Study/CSC2516/CSC2516_Final_Project/data_subset/'

batch_size = 10
# train and val only
loader = DataLoaderSegmentation(path, "train")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
inside
len:  83
image_file_size:  83


In [42]:
image_datasets = {x: DataLoaderSegmentation(path, x) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}


inside
len:  83
image_file_size:  83
inside
len:  83
image_file_size:  83


In [43]:
dataloaders_dict

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f85a874ffa0>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7f85b9783bb0>}

In [44]:
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define hyperparameters
num_epochs = 10
batch_size = 2
learning_rate = 0.001



In [45]:


# Define model
model = deeplabv3_resnet101(pretrained=True, progress=True)

In [46]:
# Modify model to output correct number of classes
num_classes = 39
model.classifier = DeepLabHead(2048, num_classes)

# Freeze the backbone layers
for param in model.backbone.parameters():
    param.requires_grad = False

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)



In [47]:
# Train the model
model.to(device)
model.train()



DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [48]:
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(dataloaders_dict['train']):
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        print("input shape: ", inputs.shape)
        outputs = model(inputs)['out']
        loss = criterion(outputs, targets.squeeze(1))
        loss.backward()
        optimizer.step()

        # Print statistics
        if (i + 1) % 1 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloaders_dict['train'])}], Loss: {loss.item():.4f}")


input shape:  torch.Size([10, 3, 224, 224])
Epoch [1/10], Step [1/9], Loss: 3.6798
input shape:  torch.Size([10, 3, 224, 224])
Epoch [1/10], Step [2/9], Loss: 3.3797
input shape:  torch.Size([10, 3, 224, 224])
Epoch [1/10], Step [3/9], Loss: 3.0300
input shape:  torch.Size([10, 3, 224, 224])
Epoch [1/10], Step [4/9], Loss: 3.1602
input shape:  torch.Size([10, 3, 224, 224])
Epoch [1/10], Step [5/9], Loss: 2.4506
input shape:  torch.Size([10, 3, 224, 224])
Epoch [1/10], Step [6/9], Loss: 2.4515
input shape:  torch.Size([10, 3, 224, 224])
Epoch [1/10], Step [7/9], Loss: 2.6149
input shape:  torch.Size([10, 3, 224, 224])
Epoch [1/10], Step [8/9], Loss: 2.2634
input shape:  torch.Size([3, 3, 224, 224])
Epoch [1/10], Step [9/9], Loss: 1.9383
input shape:  torch.Size([10, 3, 224, 224])
Epoch [2/10], Step [1/9], Loss: 2.6888
input shape:  torch.Size([10, 3, 224, 224])
Epoch [2/10], Step [2/9], Loss: 1.9525
input shape:  torch.Size([10, 3, 224, 224])
Epoch [2/10], Step [3/9], Loss: 2.4281
input